# Machine Learning Modelling

On this notebook, the definition of the Machine Learning model to be use don this work is made. For this, a couple models are tested against data, and the one with the best performance is selected, and improved by a fine-tuning process.

## 1. Packages

In [12]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_style('darkgrid')

import datetime
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression,Lasso
import xgboost as xgb
import pickle

## 2. Support Functions

On this section, some functions that going to be used througout this notebook are defined. They essentially involve a functin for calculating errors, and another function to do a proper Cross-Validation for the temporal data.

In [13]:
def ml_error(model_name,y,y_hat):
    mae = mean_absolute_error(y,y_hat)
    mape = mean_absolute_percentage_error(y,y_hat)
    rmse = np.sqrt(mean_squared_error(y,y_hat))

    return pd.DataFrame({'Model Name' : model_name,
                         'MAE' : mae,
                         'MAPE' : mape,
                         'RMSE': rmse},index=[0])

In [14]:
def cross_validation(X_train, kfold, model_name, model, verbose=False):
    mae_list = []
    mape_list = []
    rmse_list = []

    for k in reversed(range(1,kfold)):

        if verbose:
            print("\nKFold Number: {}".format(k))

        validation_start_date = X_train['date'].max() - datetime.timedelta(days=k*6*7)
        validation_end_date = X_train['date'].max() -  datetime.timedelta(days=(k-1)*6*7)

        training_set = X_train[X_train['date'] < validation_start_date]
        validation_set = X_train[(X_train['date'] >= validation_start_date) & (X_train['date'] <= validation_end_date)]

        X_training = training_set.drop(['date','sales'],axis=1)
        y_training = training_set['sales']
        
        X_validation = validation_set.drop(['date','sales'],axis=1)
        y_validation = validation_set['sales']

        m = model.fit(X_training,y_training)
        yhat = m.predict(X_validation)

        m_result = ml_error (model_name, np.expm1(y_validation),np.expm1(yhat))

        mae_list.append(m_result['MAE'])
        mape_list.append(m_result['MAPE'])
        rmse_list.append(m_result['RMSE'])

    return pd.DataFrame({
        'Model Name' : model_name,
        'MAE' : np.round(np.mean(mae_list),2).astype(str) + ' +/- ' + np.round(np.std(mae_list),2).astype(str),
        'MAPE' : np.round(np.mean(mape_list),2).astype(str) + ' +/- ' + np.round(np.std(mape_list),2).astype(str),
        'RMSE' : np.round(np.mean(rmse_list),2).astype(str) + ' +/- ' + np.round(np.std(rmse_list),2).astype(str),
    }, index=[0])

In [15]:
def train_val_split(X_train,y_train):

    validation_start_date = X_train['date'].max() - datetime.timedelta(days=k*6*7)
    validation_end_date = X_train['date'].max() -  datetime.timedelta(days=(k-1)*6*7)

    training_set = X_train[X_train['date'] < validation_start_date]
    validation_set = X_train[(X_train['date'] >= validation_start_date) & (X_train['date'] <= validation_end_date)]

    X_training = training_set.drop(['date','sales'],axis=1)
    y_training = training_set['sales']
    
    X_validation = validation_set.drop(['date','sales'],axis=1)
    y_validation = validation_set['sales']

    return X_training, y_training, X_validation, y_validation

## 3. Data Reading

In [16]:
train_data = pd.read_pickle('../Data/prepared_train_data.pkl')
test_data = pd.read_pickle('../Data/prepared_test_data.pkl')

In [17]:
X_train = train_data.drop(['date','sales'],axis=1)
y_train = train_data['sales']

X_test = test_data.drop(['sales','date'],axis=1)
y_test = test_data['sales']

## 4. Model Selection

On this section, a couple of models are applied to data and their performance is measured by a Cross Validation process. At the end, all the models are compared and one is selected for further tuning. 

### 4.1 Mean Model (Baseline)

This first model just consist of using the mean sales as predictions, and will be used as a Baseline Model.

In [18]:
mean_df = X_test.copy()
mean_df['sales'] = y_test.copy()

preds = mean_df[['store','sales']].groupby('store').mean().reset_index().rename(columns={'sales':'predictions'})
mean_df = pd.merge(mean_df,preds,how='left',on='store')
yhat_baseline = mean_df['predictions']

In [19]:
baseline_result = ml_error('Average Model',np.expm1(y_test),np.expm1(yhat_baseline))
baseline_result['RMSE'] = np.round(baseline_result['RMSE'],2).astype(str)
baseline_result['MAPE'] = np.round(baseline_result['MAPE'],2).astype(str)
baseline_result['MAE'] = np.round(baseline_result['MAE'],2).astype(str)
baseline_result

,Model Name,MAE,MAPE,RMSE
0,Average Model,1354.8,0.21,1835.14


### 4.2 Linear Regression

In [20]:
lr = LinearRegression()
lr_result_cv = cross_validation(train_data,5,'Linear Regression',lr)
lr_result_cv

,Model Name,MAE,MAPE,RMSE
0,Linear Regression,1940.46 +/- 97.24,0.3 +/- 0.02,2735.18 +/- 194.98


### 4.3 Linear Regression Regularized

In [21]:
lrr = Lasso(alpha=0.01)
lrr_result_cv = cross_validation(train_data,5,'Linear Regression Regularized CV',lrr)
lrr_result_cv

,Model Name,MAE,MAPE,RMSE
0,Linear Regression Regularized CV,1957.62 +/- 140.52,0.29 +/- 0.0,2828.15 +/- 232.95


### 4.4 Random Forests

In [22]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
rf_result_cv = cross_validation(train_data,5,'Random Forest CV',rf)
rf_result_cv

,Model Name,MAE,MAPE,RMSE
0,Random Forest CV,746.1 +/- 132.8,0.11 +/- 0.02,1125.15 +/- 203.01


### 4.5 XGBoost

On this XGBoost model, the number of estimators receives a slight tuning to a better adaptation to the actual problem.

In [23]:
model_xgb = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    eta=0.01,
    max_depth=10,
    subsample=0.7,
    colsample_bytree=0.9,
    tree_method='gpu_hist'
)

xgb_result_cv = cross_validation(train_data,5,'XGBoost CV',model_xgb,verbose=True)
xgb_result_cv


KFold Number: 4

KFold Number: 3

KFold Number: 2

KFold Number: 1


,Model Name,MAE,MAPE,RMSE
0,XGBoost CV,942.58 +/- 96.25,0.13 +/- 0.01,1372.8 +/- 151.42


### 4.6 Performance Comparison

Finnaly, all models performances are compared below.

In [24]:
modelling_result_cv = pd.concat([baseline_result,lr_result_cv,lrr_result_cv,rf_result_cv,xgb_result_cv])
modelling_result_cv.sort_values('RMSE')

,Model Name,MAE,MAPE,RMSE
0,Random Forest CV,746.1 +/- 132.8,0.11 +/- 0.02,1125.15 +/- 203.01
0,XGBoost CV,942.58 +/- 96.25,0.13 +/- 0.01,1372.8 +/- 151.42
0,Average Model,1354.8,0.21,1835.14
0,Linear Regression,1940.46 +/- 97.24,0.3 +/- 0.02,2735.18 +/- 194.98
0,Linear Regression Regularized CV,1957.62 +/- 140.52,0.29 +/- 0.0,2828.15 +/- 232.95


Looking at the results, Random Forest seems to be the best model in terms of performance. However, outside of that, this model is not so efficient, especially in terms of size of memory. Since we're on a limited resource enviroment, for this work XGBoost will be selected as the machine learning, giving up some model performance for a better deployment of the model.

## 5. Hyperparameter Fine Tuning

On this section, the selected model is submitted to a Fine Tuning of it's hyperparameters. Despite having a variety of techniques available for this process, on this work a simple search was made with trial and error and it was found that the only parameter that improved the model was the number of estimators. So, on the code below, some different values of this parameter are put to test. 

In [25]:
param = {
    'n_estimators' : [1500,1700,2500,3000,3500],
    'eta' : [0.01,0.03],
    'max_depth' : [3,5,9],
    'subsample' : [0.1,0.5,0.7],
    'colsample_bytree' : [0.3,0.7,0.9],
    'min_child_weight' : [3,8,15]
}

MAX_EVAL = 15

In [29]:
final_result = pd.DataFrame()

for size in range(MAX_EVAL):

    hp = {k : random.sample(v,1)[0] for k,v in param.items()}
    print(hp)

    # model
    model_xgb = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=hp['n_estimators'],
        eta=hp['eta'],
        max_depth=hp['max_depth'],
        subsample=hp['subsample'],
        colsample_bytree=hp['colsample_bytree'],
        min_child_weight=hp['min_child_weight'],
        tree_method = 'gpu_hist'
    )

    result = cross_validation(train_data,5,'XGBoost Regressor',model_xgb)
    final_result = pd.concat([final_result, result])
final_result

{'n_estimators': 1700, 'eta': 0.01, 'max_depth': 9, 'subsample': 0.1, 'colsample_bytree': 0.9, 'min_child_weight': 15}
{'n_estimators': 3500, 'eta': 0.03, 'max_depth': 3, 'subsample': 0.5, 'colsample_bytree': 0.9, 'min_child_weight': 15}
{'n_estimators': 3500, 'eta': 0.01, 'max_depth': 5, 'subsample': 0.5, 'colsample_bytree': 0.7, 'min_child_weight': 3}
{'n_estimators': 3000, 'eta': 0.03, 'max_depth': 5, 'subsample': 0.5, 'colsample_bytree': 0.3, 'min_child_weight': 15}
{'n_estimators': 3000, 'eta': 0.03, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 0.9, 'min_child_weight': 3}
{'n_estimators': 1700, 'eta': 0.01, 'max_depth': 5, 'subsample': 0.5, 'colsample_bytree': 0.7, 'min_child_weight': 8}
{'n_estimators': 3500, 'eta': 0.03, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 0.9, 'min_child_weight': 3}
{'n_estimators': 3500, 'eta': 0.01, 'max_depth': 3, 'subsample': 0.1, 'colsample_bytree': 0.9, 'min_child_weight': 15}
{'n_estimators': 1700, 'eta': 0.03, 'max_depth': 5, 

,Model Name,MAE,MAPE,RMSE
0,XGBoost Regressor,860.27 +/- 94.08,0.12 +/- 0.01,1249.07 +/- 147.88
0,XGBoost Regressor,1248.89 +/- 85.64,0.18 +/- 0.01,1808.79 +/- 128.85
0,XGBoost Regressor,1116.03 +/- 84.53,0.16 +/- 0.01,1628.43 +/- 128.77
0,XGBoost Regressor,1015.37 +/- 100.97,0.14 +/- 0.01,1462.83 +/- 153.27
0,XGBoost Regressor,1299.34 +/- 88.02,0.18 +/- 0.01,1884.2 +/- 131.31
0,XGBoost Regressor,1381.97 +/- 90.77,0.2 +/- 0.0,2018.22 +/- 146.1
0,XGBoost Regressor,1255.94 +/- 89.77,0.18 +/- 0.01,1816.18 +/- 133.69
0,XGBoost Regressor,1524.61 +/- 94.57,0.22 +/- 0.0,2215.55 +/- 143.9
0,XGBoost Regressor,1004.63 +/- 98.28,0.14 +/- 0.01,1455.72 +/- 145.64
0,XGBoost Regressor,785.76 +/- 94.34,0.11 +/- 0.01,1136.11 +/- 149.75


### 5.1 Final Model

On this final subsection, the tuned model is applied to test data, generating it's predictions and obtaining a final performance estimation.

In [30]:
param_tuned = {
    'n_estimators': 1700,
    'eta': 0.01,
    'max_depth': 9,
    'subsample': 0.1,
    'colsample_bytree': 0.9,
    'min_child_weight': 15
}

In [31]:
model_xgb_tuned = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=param_tuned['n_estimators'],
    eta=param_tuned['eta'],
    max_depth=param_tuned['max_depth'],
    subsample=param_tuned['subsample'],
    colsample_bytree=param_tuned['colsample_bytree'],
    min_child_weight=param_tuned['min_child_weight'],
    tree_method = 'gpu_hist').fit(X_train, y_train)

yhat_xgb_tuned = model_xgb_tuned.predict(X_test)

xgb_result_tuned = ml_error('XGBoost Regressor Tuned', np.expm1(y_test), np.expm1(yhat_xgb_tuned))
xgb_result_tuned

,Model Name,MAE,MAPE,RMSE
0,XGBoost Regressor Tuned,752.151404,0.111819,1095.760419


## 6. Model and Predictions Exporting

Finally, the model and his predictions are exported for further use.

In [34]:
pickle.dump(model_xgb_tuned, open('../Model/model_rossmann.pkl','wb'))

In [35]:
model_predictions = pd.DataFrame(yhat_xgb_tuned,columns=['Predictions'])
model_predictions.head()

,Predictions
0,8.697638
1,8.774541
2,9.136545
3,9.355741
4,8.769717


In [37]:
model_predictions.to_pickle('../Data/model_predictions.pkl')